In [ ]:
%pip install -U ultralytics

In [ ]:
# Import necessary libraries
import torch
from PIL import Image
from IPython.display import display
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load custom YOLOv5 model
model_path = 'best.pt'  # Update with your model path

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

In [ ]:
def test_model_on_video(video_path, output_video_path, threshold1, threshold2, movement_threshold=8000):
    # Open video
    cap = cv2.VideoCapture(video_path)

    # Get the width and height of the frames
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))

    # List to store the center points and heights
    center_points = []
    heights = []
    height_diffs = []

    # Define the ROI coordinates (x, y, width, height)
    roi_x = 250
    roi_y = 200
    roi_width = 500
    roi_height = frame_height - roi_y

    # Create the background subtractor object
    object_detector = cv2.createBackgroundSubtractorMOG2()

    # Define kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    initial_height = None  # Variable to store the initial height

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform inference (Placeholder - replace with actual inference code)
        results = model(frame_rgb)  # Replace with actual model inference

        # Extract bounding box coordinates and calculate center points and heights
        max_area = 0
        max_box = None
        for result in results.xyxy[0]:  # xyxy format
            x_min, y_min, x_max, y_max, confidence, class_id = result.tolist()
            area = (x_max - x_min) * (y_max - y_min)
            if area > max_area:
                max_area = area
                max_box = (x_min, y_min, x_max, y_max)

        if max_box:
            x_min, y_min, x_max, y_max = [int(coord) for coord in max_box]
            center_x = (x_min + x_max) / 2
            center_y = (y_min + y_max) / 2
            height = y_max - y_min

            # Set the initial height if it's not already set
            if initial_height is None:
                initial_height = height

            # Append data to the lists
            center_points.append((center_x, center_y))
            heights.append(height)

            # Calculate height difference from the initial height
            height_diff = abs(height - initial_height)
            height_diffs.append(height_diff)

            # Apply Gaussian blur to reduce noise
            blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)

            # Apply the background subtractor to get the mask
            mask = object_detector.apply(blurred_frame)

            # Apply morphological operations to reduce noise
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

            # Extract the ROI from the mask
            roi_mask = mask[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]

            # Count the number of non-zero pixels in the ROI mask
            movement = cv2.countNonZero(roi_mask)

            # Determine if there is significant movement in the ROI
            movement_detected = movement > movement_threshold

            # Determine state based on thresholds
            if height_diff > threshold2 and movement_detected:
                state = 'working'
            if height_diff > threshold1 and movement_detected:
                state = 'moving'
            # if height_diff <= threshold1 or movement_detected:
            # if height_diff <= threshold1 and movement_detected:
            else:
                state = 'idle'

            # Get the position for the text on the right side
            text_x = frame_width - 200  # Adjust as needed for positioning

            # Annotate the state and probabilities on the frame
            cv2.putText(frame, f'Movement: {movement_detected}', (text_x, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f'State: {state}', (text_x, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

            # Draw the center point on the frame
            cv2.circle(frame, (int(center_x), int(center_y)), 5, (0, 0, 255), -1)
            # Annotate the height on the frame
            cv2.putText(frame, f'Height: {int(height)}', (int(center_x), int(center_y - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

            # Draw the ROI rectangle on the frame
            cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (255, 0, 0), 2)

        # Write the frame into the output video
        out.write(frame)

        # Display the frame for debugging purposes
        # cv2_imshow(frame)

        key = cv2.waitKey(30)
        if key == 27:  # ESC key to exit
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Plotting the tracked center points, heights, and height differences
    if center_points:
        tracked_centers_x = [pt[0] for pt in center_points]
        tracked_centers_y = [pt[1] for pt in center_points]
        tracked_heights = heights
        tracked_height_diffs = height_diffs

        plt.figure(figsize=(18, 6))

        # Plotting center points
        plt.subplot(1, 3, 1)
        plt.plot(tracked_centers_x, tracked_centers_y, marker='o')
        plt.title('Movement of Center Points')
        plt.xlabel('Frame')
        plt.ylabel('Center Point (x, y)')

        # Plotting heights
        plt.subplot(1, 3, 2)
        plt.plot(tracked_heights, marker='o')
        plt.title('Heights of Bounding Boxes')
        plt.xlabel('Frame')
        plt.ylabel('Height')

        # Plotting height differences
        plt.subplot(1, 3, 3)
        plt.plot(tracked_height_diffs, marker='o')
        plt.title('Height Differences')
        plt.xlabel('Frame')
        plt.ylabel('Height Difference')

        plt.show()

In [ ]:
test_model_on_video('input_video/video1.mp4','output_video/processed-video1.mp4',2,10,7500)